In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install xlrd

In [ ]:
!pip install openpyxl

In [ ]:
df_in_time = pd.read_csv('../input/hr-analytics-case-study/in_time.csv')
df_general_data = pd.read_csv('../input/hr-analytics-case-study/general_data.csv')
df_out_time = pd.read_csv('../input/hr-analytics-case-study/out_time.csv')
df_manager_survey_data = pd.read_csv('../input/hr-analytics-case-study/manager_survey_data.csv')
df_employee_survey_data = pd.read_csv('../input/hr-analytics-case-study/employee_survey_data.csv')
df_data_dictionary = pd.read_excel('../input/data-dictionary/data_dictionary2.xlsx')

In [ ]:
df_out_time.rename(columns = {'Unnamed: 0':'EmployeeID'}, inplace = True)
df_in_time.rename(columns = {'Unnamed: 0':'EmployeeID'}, inplace = True)

In [ ]:
df_out_time.columns

In [ ]:
df_manager_survey_data.columns

In [ ]:
df_employee_survey_data.columns

In [ ]:
df_in_time.columns

In [ ]:
df_general_data.columns

In [ ]:
df_data_dictionary.columns

In [ ]:
!pip install openpyxl

In [ ]:
df_general_data.head()

In [ ]:
df_out_time.head()

In [ ]:
df_employee_survey_data.head()

In [ ]:
df_manager_survey_data.head()

In [ ]:
df_data_dictionary.head()

In [ ]:
df_in_time.head()

In [ ]:
df_data_dictionary.head()

Both the In_time and out_timr dataframes will help in unravelling the most important factors , the number of work hours .

In [ ]:
df_in_time[df_in_time.columns[1:262]] = df_out_time[df_out_time.columns[1:262]].astype('datetime64[ns]').sub(df_in_time[df_in_time.columns[1:262]].astype('datetime64[ns]'))

In [ ]:
df_in_time.head()

In [ ]:
df_in_time[df_in_time.columns[1:262]] = (df_in_time[df_in_time.columns[1:262]]/np.timedelta64(1, 'h'))

In [ ]:
df_in_time = df_in_time.fillna(0.0)

In [ ]:
df_in_time.head()

In [ ]:
df_in_time['Average'] = df_in_time[df_in_time.columns[1:262]].sum(axis=1)/263

In [ ]:
df_wrk_hrs = df_in_time[['EmployeeID','Average']]

In [ ]:
df_wrk_hrs.head()

In [ ]:
del df_in_time

In [ ]:
employee_data = pd.merge(df_wrk_hrs,df_employee_survey_data,left_on='EmployeeID', right_on='EmployeeID')

In [ ]:
employee_data.rename(columns = {'Average':'Emp_avg_wrkhrs'}, inplace = True)

In [ ]:
employee_data = pd.merge(df_manager_survey_data,employee_data,left_on='EmployeeID', right_on='EmployeeID')
employee_data = pd.merge(df_general_data,employee_data,left_on='EmployeeID', right_on='EmployeeID')

In [ ]:
employee_data.info()

In [ ]:
employee_data = employee_data.dropna()

In [ ]:
employee_data.head()

In [ ]:
employee_data.Attrition.unique()

In [ ]:
employee_data.BusinessTravel.unique()

In [ ]:
employee_data.Department.unique()

In [ ]:
employee_data.EducationField.unique()

In [ ]:
employee_data.Gender.unique()

In [ ]:
employee_data.MaritalStatus.unique()

In [ ]:
employee_data.Over18.unique()

In [ ]:
employee_data.StandardHours.unique()

In [ ]:
employee_data.EmployeeCount.unique()

In [ ]:
employee_data.JobSatisfaction.unique()

In [ ]:
#we need to drop attribute (EmployeeCount,Over18,StandardHours) it's not useful

employee_data = employee_data.drop(['EmployeeCount'],axis=1)
employee_data = employee_data.drop(['Over18'],axis=1)
employee_data = employee_data.drop(['StandardHours'],axis=1)

In [ ]:
employee_data.Attrition = employee_data.Attrition.map({'No': 1 , 'Yes' : 2})
employee_data.BusinessTravel = employee_data.BusinessTravel.map({'Travel_Rarely' : 1 , 'Travel_Frequently' : 2, 'Non-Travel' : 0})
employee_data.Department = employee_data.Department.map({'Sales' : 1 , 'Research & Development' : 2 ,'Human Resources': 3})
employee_data.EducationField = employee_data.EducationField.map({'Life Sciences': 1, 'Other': 2, 'Medical' : 3, 'Marketing' : 4 ,'Technical Degree': 5 , 'Human Resources' : 6 })
employee_data.Gender = employee_data.Gender.map({'Female' : 1 , 'Male' : 0})
employee_data.MaritalStatus = employee_data.MaritalStatus.map({'Married' : 2, 'Single': 1, 'Divorced':0})
employee_data.JobRole = employee_data.JobRole.map({'Healthcare Representative': 1, 'Research Scientist' : 1,
       'Sales Executive' : 2, 'Human Resources': 3, 'Research Director': 4,
       'Laboratory Technician': 4, 'Manufacturing Director': 4,
       'Sales Representative': 5, 'Manager': 5})

In [ ]:
sns.violinplot(x='JobSatisfaction',y='Attrition', orient='h',data=employee_data)

In [ ]:
sns.violinplot(x='JobSatisfaction',y='JobInvolvement', orient='h',data=employee_data)

In [ ]:
sns.violinplot(x='JobSatisfaction',y='PerformanceRating', orient='h',data=employee_data)

In [ ]:
sns.displot(employee_data.Emp_avg_wrkhrs)
fig = plt.figure()
res = stats.probplot(employee_data.Emp_avg_wrkhrs , plot=plt)

In [ ]:
sns.displot(employee_data.JobSatisfaction)

In [ ]:
employee_data.columns

In [ ]:
sns.displot(employee_data.JobInvolvement)

In [ ]:
sns.displot(employee_data.EnvironmentSatisfaction)

In [ ]:
sns.displot(employee_data.JobSatisfaction)

In [ ]:
sns.displot(employee_data.WorkLifeBalance)

In [ ]:
employee_data.head()

In [ ]:
sns.displot(employee_data.BusinessTravel)

In [ ]:
sns.violinplot(y='BusinessTravel',x='JobSatisfaction', orient='h',data=employee_data)

In [ ]:
#correlation matrix
corrmat = employee_data.corr()
f, ax = plt.subplots(figsize=(24, 18))
sns.heatmap(corrmat, vmax=.8,cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10});

# Assumption applied  

<h4> We will consider two attributes to be proportinal only when the correlation matrix values is >= 50.

# Conclusion drawn from Correlation Matrix :

* **Age is proportional to the number of year worked in** : 
 The older you are the the more you have been working.
 
* **Percentage Salary hike is proportinal to performance rating** :
The people who have good rating get paid better.

* **TotalNumberWorkingYears of is directly proportional to YearsAtCompany** :
 The older you are the the more you have been working.
 
* **YearsAtCompany is directly proportional to YearsWithCurrManager** :
This means the employees never get reassigned 

* **YearsSinceLastPromotion is directly proportional to YearsWithCurrManager** :
This means the employees never get promoted maybe the manager is the reason and they don't let people get promoted.

* **YearsAtCompany is directly proportional to YearsSinceLastPromotion** :
 The employee are not promoted based on experience and the number of years they have contributed .
 



In [ ]:
employee_data.shape

In [ ]:
#JobSatisfaction correlation matrix


k = 7 #number of variables for heatmap
corrmat = employee_data.corr()
cols = corrmat.nlargest(k, 'JobSatisfaction')['JobSatisfaction'].index
cm = np.corrcoef(employee_data[cols].values.T)
cm = np.nan_to_num(cm)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=False, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Job Satisfaction is not correlated to the following :

'Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
'Education', 'EducationField', 'EmployeeID', 'Gender', 'JobLevel',
'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked',
'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears',
'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
'YearsWithCurrManager', 'JobInvolvement', 'PerformanceRating',
'Emp_avg_wrkhrs', 'EnvironmentSatisfaction', 'JobSatisfaction',
'WorkLifeBalance'

In [ ]:
#PerformanceRating correlation matrix
k = 7 #number of variables for heatmap
corrmat = employee_data.corr()
cols = corrmat.nlargest(k, 'PerformanceRating')['PerformanceRating'].index
cm = np.corrcoef(employee_data[cols].values.T)
cm = np.nan_to_num(cm)
sns.set(font_scale=1.25)
#f, ax = plt.subplots(figsize=(12,9))
hm = sns.heatmap(cm, cbar=False, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
#JobInvolvement correlation matrix
k = 7 #number of variables for heatmap
corrmat = employee_data.corr()
cols = corrmat.nlargest(k, 'JobInvolvement')['JobInvolvement'].index
cm = np.corrcoef(employee_data[cols].values.T)
cm = np.nan_to_num(cm)
sns.set(font_scale=1.25)
#f, ax = plt.subplots(figsize=(12,9))
hm = sns.heatmap(cm, cbar=False, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:

#TrainingTimesLastYear correlation matrix
k = 7 #number of variables for heatmap
corrmat = employee_data.corr()
cols = corrmat.nlargest(k, 'TrainingTimesLastYear')['TrainingTimesLastYear'].index
cm = np.corrcoef(employee_data[cols].values.T)
cm = np.nan_to_num(cm)
sns.set(font_scale=1.25)
#f, ax = plt.subplots(figsize=(12,9))
hm = sns.heatmap(cm, cbar=False, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:

#JobRole correlation matrix
k = 7 #number of variables for heatmap
corrmat = employee_data.corr()
cols = corrmat.nlargest(k, 'JobRole')['JobRole'].index
cm = np.corrcoef(employee_data[cols].values.T)
cm = np.nan_to_num(cm)
sns.set(font_scale=1.25)
#f, ax = plt.subplots(figsize=(12,9))
hm = sns.heatmap(cm, cbar=False, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
#employee_data.info()

# Can we use this data for other attributes ?
* JobSatisfaction,PerformanceRating,TrainingTimesLastYear,JobRole show really low correlation with the other attributes .
* Hence the these attributes cannot be determined using these data